In [33]:
import numpy as np
def conv_backward_naive(dout, cache):
  """
  A naive implementation of the backward pass for a convolutional layer.

  Inputs:
  - dout: Upstream derivatives.
  - cache: A tuple of (x, w, b, conv_param) as in conv_forward_naive

  Returns a tuple of:
  - dx: Gradient with respect to x
  - dw: Gradient with respect to w
  - db: Gradient with respect to b
  """
  dx, dw, db = None, None, None

  x, w, b = cache
  N,C,H,W = x.shape
  F,C,HH,WW = w.shape
  H_prime = (H-HH) + 1
  W_prime = (W-WW) + 1

  dw = np.zeros(w.shape)
  dx = np.zeros(x.shape)
  db = np.zeros(b.shape)

  # We could calculate the bias by just summing over the right dimensions
  # Bias gradient (Sum on dout dimensions (batch, rows, cols)
  #db = np.sum(dout, axis=(0, 2, 3))

  for i in range(N):
      im_pad = x[i,:,:,:]
      im_col = im2col(im_pad,HH,WW,1)
      filter_col = np.reshape(w,(F,-1)).T

      dout_i = dout[i,:,:,:]
      dbias_sum = np.reshape(dout_i,(F,-1))
      dbias_sum = dbias_sum.T

      #bias_sum = mul + b
      db += np.sum(dbias_sum,axis=0)
      dmul = dbias_sum

      #mul = im_col * filter_col
      dfilter_col = (im_col.T).dot(dmul)
      print(dmul.shape, filter_col.T.shape)
      dim_col = dmul.dot(filter_col.T)

      dx_padded = col2im_back(dim_col,H_prime,W_prime,1,HH,WW,C)
      dx[i,:,:,:] = dx_padded[:,0:H,0:W]
      dw += np.reshape(dfilter_col.T,(F,C,HH,WW))
  return dx, dw, db


In [49]:
def im2col(x,hh,ww,stride):

    """
    Args:
      x: image matrix to be translated into columns, (C,H,W)
      hh: filter height
      ww: filter width
      stride: stride
    Returns:
      col: (new_h*new_w,hh*ww*C) matrix, each column is a cube that will convolve with a filter
            new_h = (H-hh) // stride + 1, new_w = (W-ww) // stride + 1
    """

    c,h,w = x.shape
    new_h = (h-hh) // stride + 1
    new_w = (w-ww) // stride + 1
    col = np.zeros([new_h*new_w,c*hh*ww])

    for i in range(new_h):
       for j in range(new_w):
           patch = x[...,i*stride:i*stride+hh,j*stride:j*stride+ww]
           col[i*new_w+j,:] = np.reshape(patch,-1)
    return col

def col2im(mul,h_prime,w_prime,C):
    """
      Args:
      mul: (h_prime*w_prime*w,F) matrix, each col should be reshaped to C*h_prime*w_prime when C>0, or h_prime*w_prime when C = 0
      h_prime: reshaped filter height
      w_prime: reshaped filter width
      C: reshaped filter channel, if 0, reshape the filter to 2D, Otherwise reshape it to 3D
    Returns:
      if C == 0: (F,h_prime,w_prime) matrix
      Otherwise: (F,C,h_prime,w_prime) matrix
    """
    F = mul.shape[1]
    if(C == 1):
        out = np.zeros([F,h_prime,w_prime])
        for i in range(F):
            col = mul[:,i]
            out[i,:,:] = np.reshape(col,(h_prime,w_prime))
    else:
        out = np.zeros([F,C,h_prime,w_prime])
        for i in range(F):
            col = mul[:,i]
            out[i,:,:] = np.reshape(col,(C,h_prime,w_prime))

    return out

def col2im_back(dim_col,h_prime,w_prime,stride,hh,ww,c):
    """
    Args:
      dim_col: gradients for im_col,(h_prime*w_prime,hh*ww*c)
      h_prime,w_prime: height and width for the feature map
      strid: stride
      hh,ww,c: size of the filters
    Returns:
      dx: Gradients for x, (C,H,W)
    """
    print(h_prime)
    print(w_prime)
    print(hh)
    print(ww)
    H = int((h_prime - 1) * stride + hh)
    W = int((w_prime - 1) * stride + ww)
    dx = np.zeros([c,H,W])
    print(dx.shape)
    for i in range(h_prime*w_prime):
        row = dim_col[i,:]
        h_start = int((i / w_prime) * stride)
        w_start = int((i % w_prime) * stride)
        print(h_start, w_start)
        dx[:,h_start:h_start+hh,w_start:w_start+ww] += np.reshape(row,(c,hh,ww))
    print(dx.shape)
    return dx

In [50]:
cache = dict()
cache['x'] = np.random.randn(2, 6, 13, 13)
cache['w'] = np.random.randn(16, 6, 3, 3)
cache['b'] = np.random.randn(16)
cache = (cache['x'], cache['w'], cache['b'])
dout = np.random.randn(2, 16, 11, 11)
dx, dw, db = conv_backward_naive(dout, cache)

(121, 16) (16, 54)
11
11
3
3
(6, 13, 13)
0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
7 10
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
8 10
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9
9 10
10 0
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
10 10
(6, 13, 13)
(121, 16) (16, 54)
11
11
3
3
(6, 13, 13)
0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
7 10
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9


In [17]:
dx

array([[[[ -3.36827113,  -0.65160937,   7.12135642, ...,   5.84980384,
           -1.74683695,  -0.67293779],
         [  2.1693469 ,  -0.48887211,  10.3373096 , ...,  -3.53481662,
           16.31029144,  -3.29171516],
         [ -0.06495796,  -7.75884848,  -3.57374069, ...,  12.6859035 ,
           26.63869245,  10.35722145],
         ...,
         [  2.98768258,  -4.54868289, -11.7673802 , ...,   2.13664031,
            1.23112468,  12.48650446],
         [ -1.38737368,   7.56916801,   5.70164278, ...,   5.41080627,
           11.49004462,   1.35352731],
         [  7.25921345,   4.64831587,   2.67893909, ...,  -1.75974035,
            5.21661409,   0.03039314]],

        [[ -4.24182066,   8.83162925,  -9.65781209, ...,   4.76023625,
           -7.03690438,  -2.17598465],
         [  0.75796496,  -6.24083929, -17.57993266, ...,   3.89014721,
            6.42156898,  -2.22227202],
         [ -7.97511166, -14.58480416,  -5.58985523, ...,   9.9890899 ,
          -14.03607268,  -2.30362